In [4]:
!pip install qiskit

  Obtaining dependency information for qiskit from https://files.pythonhosted.org/packages/36/43/be9bc635ff62d27a18de52f87aaaf7eb7b0bc80f1241684c4673907d5757/qiskit-0.44.1-py3-none-any.whl.metadata
  Obtaining dependency information for qiskit-terra==0.25.1 from https://files.pythonhosted.org/packages/28/38/137a26c022fb24234aafe08ab764c0611c090ce1715bd66238b629461184/qiskit_terra-0.25.1-cp38-abi3-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for rustworkx>=0.13.0 from https://files.pythonhosted.org/packages/c3/6e/600b4cb2378a34d9e12b9cb8f0e2640aa86f34b3fefce30093caa88c9010/rustworkx-0.13.1-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Using cached rustworkx-0.13.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (10 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Obtaining dependency information for dill>=0.3 from https://files.pythonhosted.org/packages/f5/3a/74a29b11cf2cdfcd6ba89c0cecd70b37cd1ba7b77978ce611eb7a146a832/dill-0.3.7-py3-none-any.whl.metadata
  Using cac

In [5]:
!pip install qiskit-aer

  Obtaining dependency information for qiskit-aer from https://files.pythonhosted.org/packages/31/8a/1d33d601eb725aa3d76689e02437dc9ef177d46687a9ddbf3c4e29a304a3/qiskit_aer-0.12.2-cp39-cp39-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 83.4 kB/s eta 0:00:0000:0100:01m


In [9]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to your train data folders
train_data_dir = 'train'

# Set image dimensions and batch size
img_width, img_height = 128, 128
batch_size = 32

# Create an ImageDataGenerator for data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generate training data
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    color_mode='rgb'
)

# Extract X_train (input images) and Y_train (labels)
X_train = []
Y_train = []

for images, labels in train_generator:
    X_train.extend(images)
    Y_train.extend(labels)
    
    if len(X_train) >= len(train_generator.filenames):
        break

X_train = np.array(X_train)
Y_train = np.array(Y_train)

# Print class labels and their corresponding indices
class_indices = train_generator.class_indices
print("Class Indices:", class_indices)

# Convert labels to one-hot encoding
num_classes = len(class_indices)
Y_train_one_hot = tf.keras.utils.to_categorical(Y_train, num_classes=num_classes)

print("X_train shape:", X_train.shape)
print("Y_train_one_hot shape:", Y_train_one_hot.shape)


Found 28709 images belonging to 7 classes.
Class Indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
X_train shape: (28709, 128, 128, 3)
Y_train_one_hot shape: (28709, 7, 7)


In [12]:
import os
import numpy as np
import tensorflow as tf
import qiskit
from qiskit import QuantumCircuit, Aer
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import StateFn
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Classical Convolutional Neural Network (CNN)
def create_classical_cnn(input_shape):
    model = tf.keras.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten()
    ])
    return model

# Quantum Circuit for embedding (using Qiskit's classical component)
def create_quantum_embedding_model(qubits):
    backend = Aer.get_backend('statevector_simulator')
    qc = QuantumCircuit(qubits)
    ry_layer = RealAmplitudes(num_qubits=qubits, entanglement='full', reps=1)
    ry_circuit = ry_layer.bind_parameters({param: 0.01 for param in ry_layer.parameters})
    psi = StateFn(ry_circuit)
    embedding = psi.compose(StateFn(qc))
    result = backend.run(embedding).result()
    embedding_vector = result.get_statevector()
    return embedding_vector

# Create the hybrid model
def create_hybrid_mood_detection_model(input_shape, qubits, num_moods):
    classical_cnn = create_classical_cnn(input_shape)
    quantum_input = create_quantum_embedding_model(qubits)

    # Merge classical and quantum features
    merged_features = Concatenate()([classical_cnn.output, quantum_input])

    # Add dense layers for mood classification
    dense_layer = Dense(128, activation='relu')(merged_features)
    output_layer = Dense(num_moods, activation='softmax')(dense_layer)

    model = tf.keras.Model(inputs=[classical_cnn.input], outputs=output_layer)
    return model

# Define input data shapes
input_shape = (img_width, img_height, 3)  # Adjust based on your photo dimensions
num_moods = len(train_generator.class_indices)

# Create the hybrid mood detection model
qubits = 4  # Number of qubits for the quantum circuit
hybrid_model = create_hybrid_mood_detection_model(input_shape, qubits, num_moods)

# Compile the model
hybrid_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
hybrid_model.summary()

# Train the hybrid model using the training data
num_epochs = 10
steps_per_epoch = len(train_generator) // batch_size

hybrid_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs
)

/var/folders/mh/v3x6dz8534gcdj91_hh0bdph0000gn/T/ipykernel_37266/4063958212.py:31: DeprecationWarning: The class ``qiskit.opflow.state_fns.circuit_state_fn.CircuitStateFn`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  psi = StateFn(ry_circuit)
/var/folders/mh/v3x6dz8534gcdj91_hh0bdph0000gn/T/ipykernel_37266/4063958212.py:32: DeprecationWarning: The class ``qiskit.opflow.state_fns.circuit_state_fn.CircuitStateFn`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  embedding = psi.compose(StateFn(qc))


ValueError: Composition with a Statefunctions in the first operand is not defined.